In [59]:
import pandas as pd
from pandas.plotting import table
import requests
import json
from config import key, bkey
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cbook, ticker, units
import datetime
import functools
import re

## US Census Data API

In [60]:
census = pd.read_html('https://api.census.gov/data.html')[0]

In [61]:
list(census["Title"])

['1986 County Business Patterns: Business Patterns',
 '1987 County Business Patterns: Business Patterns',
 '1988 County Business Patterns: Business Patterns',
 '1989 County Business Patterns: Business Patterns',
 '1990 County Business Patterns: Business Patterns',
 '1990 Population Estimates - 1990-2000 Intercensal Estimates: County Estimates by 5 Year Age Groups, Race by Sex, and Hispanic Origin',
 '1990 Population Estimates - 1990-2000 Intercensal Estimates: United States Civilian Population Estimates by Age and Sex',
 '1990 Population Estimates - 1990-2000 Intercensal Estimates: United States Resident plus Armed Forces Overseas Population Estimates by Age and Sex',
 '1990 Population Estimates - 1990-2000 Intercensal Estimates: United States Resident Population Estimates by Age and Sex',
 '1991 County Business Patterns: Business Patterns',
 '1992 County Business Patterns: Business Patterns',
 '1993 County Business Patterns: Business Patterns',
 '1994 County Business Patterns: Busines

### International Trade

In [62]:
trade_census = census[census["Title"].str.contains("International Trade")]
pd.set_option('display.max_colwidth', None)
trade_census

,Title,Description,Vintage,Dataset Name,Dataset Type,Geography List,Variable List,Group List,Examples,Developer Documentation,API Base URL
306,2014 International Trade: Historical Imports and Exports,"This international trade file provides the annual dollar value of U.S. exports and imports of goods for all U.S. trade partners. It also provides the annual dollar value of U.S. exports and imports of manufactured goods for all U.S. trade partners. You can find this data and more by going to usatrade.census.gov. If you have any questions regarding U.S. international trade data, please call us at 1(800)549-0595 option #4 or email us at eid.international.trade.data@census.gov.",2014,intltrade› imp_exp,Aggregate,geographies,variables,groups,examples,documentation,https://api.census.gov/data/2014/intltrade/imp_exp
352,2015 International Trade: Historical Imports and Exports,"This international trade file provides the annual dollar value of U.S. exports and imports of goods for all U.S. trade partners. It also provides the annual dollar value of U.S. exports and imports of manufactured goods for all U.S. trade partners. You can find this data and more by going to usatrade.census.gov. If you have any questions regarding U.S. international trade data, please call us at 1(800)549-0595 option #4 or email us at eid.international.trade.data@census.gov.",2015,intltrade› imp_exp,Aggregate,geographies,variables,groups,examples,documentation,https://api.census.gov/data/2015/intltrade/imp_exp
378,2016 International Trade: Historical Imports and Exports,"This international trade file provides the annual dollar value of U.S. exports and imports of goods for all U.S. trade partners. It also provides the annual dollar value of U.S. exports and imports of manufactured goods for all U.S. trade partners. You can find this data and more by going to usatrade.census.gov. If you have any questions regarding U.S. international trade data, please call us at 1(800)549-0595 option #4 or email us at eid.international.trade.data@census.gov.",2016,intltrade› imp_exp,Aggregate,geographies,variables,groups,examples,documentation,https://api.census.gov/data/2016/intltrade/imp_exp
461,2017 International Trade: Historical Imports and Exports,"This international trade file provides the annual dollar value of U.S. exports and imports of goods for all U.S. trade partners. It also provides the annual dollar value of U.S. exports and imports of manufactured goods for all U.S. trade partners. You can find this data and more by going to usatrade.census.gov. If you have any questions regarding U.S. international trade data, please call us at 1(800)549-0595 option #4 or email us at eid.international.trade.data@census.gov.",2017,intltrade› imp_exp,Aggregate,geographies,variables,groups,examples,documentation,https://api.census.gov/data/2017/intltrade/imp_exp
492,2018 International Trade: Historical Imports and Exports,"This international trade file provides the annual dollar value of U.S. exports and imports of goods for all U.S. trade partners. It also provides the annual dollar value of U.S. exports and imports of manufactured goods for all U.S. trade partners. You can find this data and more by going to usatrade.census.gov. If you have any questions regarding U.S. international trade data, please call us at 1(800)549-0595 option #4 or email us at eid.international.trade.data@census.gov.",2018,intltrade› imp_exp,Aggregate,geographies,variables,groups,examples,documentation,https://api.census.gov/data/2018/intltrade/imp_exp
537,Time Series Economic Indicators Time Series -: U.S. International Trade in Goods and Services,"The U.S. Census Bureau.s economic indicator surveys provide monthly and quarterly data that are timely, reliable, and offer comprehensive measures of the U.S. economy. These surveys produce a variety of statistics covering construction, housing, international trade, retail trade, wholesale trade, services and manufacturing. The survey data provide measures of 

# Import

### Time Series International Trade: Monthly U.S. Imports by Port and Harmonized System (HS) Code<br>
The Port HS endpoint in the Census data API also provides value, shipping weight, and method of transportation totals at the port level for all U.S. trading partners. The Census data API will help users research new markets for their products, establish pricing structures for potential export markets, and conduct economic planning.

#### Variables

In [63]:
portVar = pd.read_html('https://api.census.gov/data/timeseries/intltrade/imports/porths/variables.html')[0]
portVar

,Name,Label,Concept,Required,Attributes,Limit,Predicate Type,Group,Unnamed: 8
0,AIR_VAL_MO,15-digit Air Value,Measures,not required,NaN,0,int,NaN,NaN
1,AIR_VAL_YR,15-digit Year-to-Date Air Value,Measures,not required,NaN,0,int,NaN,NaN
2,AIR_WGT_MO,15-digit Air Shipping Weight,Measures,not required,NaN,0,int,NaN,NaN
3,AIR_WGT_YR,15-digit Year-to-Date Air Shipping Weight,Measures,not required,NaN,0,int,NaN,NaN
4,CNT_VAL_MO,15-digit Containerized Vessel Value,Measures,not required,NaN,0,int,NaN,NaN
5,CNT_VAL_YR,15-digit Year-to-Date Containerized Vessel Value,Measures,not required,NaN,0,int,NaN,NaN
6,CNT_WGT_MO,15-digit Containerized Vessel Shipping Weight,Measures,not required,NaN,0,int,NaN,NaN
7,CNT_WGT_YR,15-digit Year-to-Date Containerized Vessel Shipping Weight,Measures,not required,NaN,0,int,NaN,NaN
8,COMM_LVL,4-character aggregation levels for commodity code. HS2=2-digit HS totals. HS4=4-digit HS totals. HS6=6-digit HS totals.,Measures,not required,NaN,0,string,NaN,NaN
9,CTY_CODE,4-character Country Code,Measures,default displayed,NaN,0,string,NaN,NaN


In [64]:
portDict = dict(zip(portVar["Name"], portVar["Label"].str.replace('15-digit ', '')))

## Port and Container Count

In [65]:
portURL = 'https://api.census.gov/data/timeseries/intltrade/imports/porths?get='
portURLreq = requests.get(f"{portURL}PORT,PORT_NAME,CNT_WGT_MO&time=from+2019+to+2021&key={key}")
print(portURLreq.url)

https://api.census.gov/data/timeseries/intltrade/imports/porths?get=PORT,PORT_NAME,CNT_WGT_MO&time=from+2019+to+2021&key=c140aee8a215a9f809ef67c3c758614e79362587


In [66]:
portData = portURLreq.json()
portDF = pd.DataFrame(portData, columns=portData[0]).drop([0]).rename(columns=portDict)
pd.set_option('display.max_colwidth', None)
portDF

,4-character Port Code,150-character Port Name,Containerized Vessel Shipping Weight,ISO-8601 Date/Time value
1,-,TOTAL FOR ALL PORTS,15840254477,2019-01
2,-,TOTAL FOR ALL PORTS,13329969180,2019-02
3,-,TOTAL FOR ALL PORTS,14054279566,2019-03
4,-,TOTAL FOR ALL PORTS,14721360090,2019-04
5,-,TOTAL FOR ALL PORTS,15441730719,2019-05
...,...,...,...,...
9367,2401,"YSLETA, TX",0,2020-11
9368,2401,"YSLETA, TX",0,2020-12
9369,2401,"YSLETA, TX",0,2021-01
9370,2401,"YSLETA, TX",0,2021-02


In [67]:
port = portDF[~portDF['150-character Port Name'].str.contains("TOTAL")].drop(columns=['4-character Port Code']).rename(columns={'150-character Port Name':'Port','ISO-8601 Date/Time value':'Date'})
#port['Date'] = pd.to_datetime(port['Date'])
port.head()

,Port,Containerized Vessel Shipping Weight,Date
28,"PORTLAND, ME",8582227,2019-01
29,"PORTLAND, ME",14823829,2019-02
30,"PORTLAND, ME",12000667,2019-03
31,"PORTLAND, ME",19453602,2019-04
32,"PORTLAND, ME",14481568,2019-05


In [68]:
port.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9344 entries, 28 to 9371
Data columns (total 3 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Port                                  9344 non-null   object
 1   Containerized Vessel Shipping Weight  9344 non-null   object
 2   Date                                  9344 non-null   object
dtypes: object(3)
memory usage: 292.0+ KB


In [69]:
portCOL = port.columns.drop(['Port',"Date"])
port[portCOL] = port[portCOL].apply(pd.to_numeric, errors='coerce')
port.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9344 entries, 28 to 9371
Data columns (total 3 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Port                                  9344 non-null   object
 1   Containerized Vessel Shipping Weight  9344 non-null   int64 
 2   Date                                  9344 non-null   object
dtypes: int64(1), object(2)
memory usage: 292.0+ KB


In [70]:
port.head()

,Port,Containerized Vessel Shipping Weight,Date
28,"PORTLAND, ME",8582227,2019-01
29,"PORTLAND, ME",14823829,2019-02
30,"PORTLAND, ME",12000667,2019-03
31,"PORTLAND, ME",19453602,2019-04
32,"PORTLAND, ME",14481568,2019-05


Consolidate Port by State

In [71]:
list(port['Port'].unique())

['PORTLAND, ME',
 'BANGOR, ME',
 'EASTPORT, ME',
 'JACKMAN, ME',
 'VANCEBORO, ME',
 'HOULTON, ME',
 'FORT FAIRFIELD, ME',
 'VAN BUREN, ME',
 'MADAWASKA, ME',
 'FORT KENT, ME',
 'BATH, ME',
 'BAR HARBOR, ME',
 'CALAIS, ME',
 'LIMESTONE, ME',
 'ROCKLAND, ME',
 'JONESPORT, ME',
 'BRIDGEWATER, ME',
 'PORTSMOUTH, NH',
 'BELFAST, ME',
 'SEARSPORT, ME',
 'MANCHESTER USER FEE AIRPORT, NH',
 'ST. ALBANS, VT',
 'RICHFORD, VT',
 'BEECHER FALLS, VT',
 'BURLINGTON, VT',
 'DERBY LINE, VT',
 'NORTON, VT',
 'HIGHGATE SPRINGS-ALBURG, VT',
 'BOSTON, MA',
 'SPRINGFIELD, MA',
 'WORCESTER, MA',
 'GLOUCESTER, MA',
 'NEW BEDFORD, MA',
 'PLYMOUTH, MA',
 'FALL RIVER, MA',
 'SALEM, MA',
 'BRIDGEPORT, CT',
 'HARTFORD, CT',
 'NEW HAVEN, CT',
 'NEW LONDON, CT',
 'LAWRENCE, MA',
 'LOGAN AIRPORT, MA',
 'L.G. HANSCOM FIELD, BEDFORD, MA',
 'NEWPORT, RI',
 'PROVIDENCE, RI',
 'OGDENSBURG, NY',
 'MASSENA, NY',
 'ALEXANDRIA BAY, NY',
 'CHAMPLAIN-ROUSES POINT, NY',
 'TROUT RIVER, NY',
 'BUFFALO-NIAGARA FALLS, NY',
 'ROCHES

In [72]:
portST = port.copy()
portST['Port State'] = port['Port'].str[-2:]

In [73]:
portST.head()

,Port,Containerized Vessel Shipping Weight,Date,Port State
28,"PORTLAND, ME",8582227,2019-01,ME
29,"PORTLAND, ME",14823829,2019-02,ME
30,"PORTLAND, ME",12000667,2019-03,ME
31,"PORTLAND, ME",19453602,2019-04,ME
32,"PORTLAND, ME",14481568,2019-05,ME


In [74]:
portST = portST.groupby(['Date','Port State']).sum().reset_index().rename(columns={'Containerized Vessel Shipping Weight':'Weight'})
portST

,Date,Port State,Weight
0,2019-01,AK,3377105
1,2019-01,AL,116945198
2,2019-01,AR,0
3,2019-01,AZ,0
4,2019-01,CA,5217244404
...,...,...,...
1453,2021-03,VI,1033957
1454,2021-03,VT,0
1455,2021-03,WA,905977208
1456,2021-03,WI,73876


In [75]:
cleanPort = portST[(portST.Weight != 0)]
cleanPort

,Date,Port State,Weight
0,2019-01,AK,3377105
1,2019-01,AL,116945198
4,2019-01,CA,5217244404
6,2019-01,CT,10767755
7,2019-01,DC,637104
...,...,...,...
1449,2021-03,TX,1283579816
1452,2021-03,VA,901597179
1453,2021-03,VI,1033957
1455,2021-03,WA,905977208


### Estimate TEU Count (Twenty Equivalent Count) = 1 Twenty Foot Container

Estimate Container count by dividing Weight(kg) to tons. <br>
1 TEU = 24,000 kg max load weight

In [129]:
portTEU = cleanPort.copy().rename(columns={'Port State':'Port'})

In [130]:
portTEU['TEU'] = portTEU['Weight'].floordiv(24000)

In [131]:
portTEU

,Date,Port,Weight,TEU
0,2019-01,AK,3377105,140
1,2019-01,AL,116945198,4872
4,2019-01,CA,5217244404,217385
6,2019-01,CT,10767755,448
7,2019-01,DC,637104,26
...,...,...,...,...
1449,2021-03,TX,1283579816,53482
1452,2021-03,VA,901597179,37566
1453,2021-03,VI,1033957,43
1455,2021-03,WA,905977208,37749


In [132]:
portTEU.to_csv("./static/data/portTEU.csv")

In [133]:
portTEU.to_json("./static/data/portTEU.json")

### Heatmap - Total Container Count by State & Date

In [134]:
portTEUdf = portTEU.pivot(index="Date", columns="Port", values="TEU")
portTEUdf.describe().astype(int)

Port,AK,AL,CA,CT,DC,DE,FL,GA,HI,IL,...,OR,PA,PR,RI,SC,TX,VA,VI,WA,WI
count,27,27,27,21,27,27,27,27,27,27,...,27,27,27,26,27,27,27,27,27,27
mean,122,5849,198596,411,18,6663,36751,53266,3277,464,...,434,15185,13625,425,28818,44674,32796,39,30086,325
std,56,1107,24656,748,9,792,3310,6520,1598,542,...,457,1952,2516,597,2685,4120,2806,29,3312,508
min,26,4267,145034,0,4,5674,32018,42652,465,40,...,2,11679,8886,0,22731,37289,26806,2,23077,1
25%,95,5046,182479,3,9,6032,34420,48233,2649,92,...,7,13929,11943,5,27121,41774,31147,12,28148,5
50%,119,5410,201561,68,19,6479,35172,52885,2928,215,...,346,14684,14315,83,28703,45063,32911,37,30023,27
75%,150,6650,218506,161,26,6992,38337,57583,4674,542,...,700,16619,15115,797,30056,47474,34499,62,31882,653
max,272,7888,238184,2157,39,8356,45391,67823,6393,2031,...,1380,21290,18066,2100,35454,53482,37582,115,37749,1673


In [147]:
portTEUdf

Port,Date,AK,AL,CA,CT,DC,DE,FL,GA,HI,...,OR,PA,PR,RI,SC,TX,VA,VI,WA,WI
0,2019-01,140.0,4872.0,217385.0,448.0,26.0,6179.0,37466.0,56873.0,2925.0,...,7.0,15897.0,14315.0,0.0,29590.0,46389.0,34020.0,84.0,34506.0,27.0
1,2019-02,166.0,4520.0,180709.0,NaN,28.0,6479.0,38235.0,42652.0,1045.0,...,14.0,12656.0,8886.0,0.0,25052.0,37289.0,31496.0,44.0,29491.0,36.0
2,2019-03,119.0,5143.0,164047.0,76.0,19.0,7556.0,38670.0,52893.0,5638.0,...,7.0,14218.0,13601.0,NaN,28678.0,51230.0,33395.0,47.0,28927.0,22.0
3,2019-04,192.0,5408.0,186709.0,68.0,13.0,7464.0,36484.0,49944.0,5070.0,...,4.0,16923.0,14734.0,929.0,28385.0,48500.0,37582.0,29.0,31650.0,10.0
4,2019-05,61.0,5104.0,207305.0,NaN,27.0,5801.0,34992.0,53432.0,2928.0,...,2.0,15380.0,9348.0,1.0,29704.0,45982.0,36015.0,44.0,35228.0,8.0
5,2019-06,198.0,4823.0,190456.0,22.0,14.0,6404.0,35049.0,47916.0,5033.0,...,3.0,13903.0,13830.0,38.0,25951.0,48266.0,32911.0,115.0,30023.0,61.0
6,2019-07,211.0,5275.0,217710.0,0.0,28.0,6660.0,35172.0,54974.0,465.0,...,9.0,17437.0,11842.0,32.0,28917.0,44572.0,36414.0,77.0,34040.0,43.0
7,2019-08,128.0,5583.0,202875.0,0.0,26.0,6578.0,34783.0,58294.0,4725.0,...,177.0,14684.0,14768.0,53.0,30554.0,48202.0,33821.0,37.0,33035.0,694.0
8,2019-09,121.0,6307.0,193146.0,0.0,31.0,5710.0,33003.0,51582.0,2778.0,...,49.0,14381.0,15447.0,2.0,28535.0,42010.0,33442.0,34.0,30863.0,6.0
9,2019-10,161.0,6956.0,190793.0,NaN,39.0,6416.0,37522.0,52885.0,3019.0,...,4.0,13525.0,16059.0,1952.0,29519.0,44310.0,34049.0,70.0,29876.0,1673.0


In [115]:
portTEUdf.to_csv("./static/data/port.csv")

In [138]:
portTEUdf.to_json("./static/data/port.json",orient="index")

In [149]:
group = portTEU.copy().rename(columns={"Port State": 'Port'})
group = group.groupby(['Date','Port']).sum().drop(columns='Weight')
group

TEU
Date    Port        
2019-01 AK       140
        AL      4872
        CA    217385
        CT       448
        DC        26
...              ...
2021-03 TX     53482
        VA     37566
        VI        43
        WA     37749
        WI         3

[875 rows x 1 columns]

In [150]:
group.to_csv("./static/data/gport.csv")

In [146]:
group.to_json("./static/data/gport.json", orient="index")

In [ ]:
import seaborn as sns

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
sns.heatmap(portTEUdf,cmap="Blues", vmin=100, vmax=250000)
plt.show()

### Time Series International Trade: Monthly U.S. Imports by End-use Code <br>
The Census data API provides access to the most comprehensive set of data on current month and cumulative year-to-date imports using the End-use classification system. The End-use endpoint in the Census data API also provides value, shipping weight, and method of transportation totals at the district level for all U.S. trading partners.

In [ ]:
useVar = pd.read_html('https://api.census.gov/data/timeseries/intltrade/imports/enduse/variables.html')[0]
useVar

In [ ]:
useDict = dict(zip(useVar["Name"], useVar["Label"].str.replace('15-digit ', '')))

In [ ]:
useURL = 'https://api.census.gov/data/timeseries/intltrade/imports/enduse?get='
useURLreq = requests.get(f"{useURL}I_ENDUSE,I_ENDUSE_LDESC,I_ENDUSE_SDESC,DISTRICT,DIST_NAME,CNT_WGT_MO,CON_VAL_MO&time=from+2019+to+2021&key={key}")
print(useURLreq.url)

In [ ]:
useData = useURLreq.json()
useDF = pd.DataFrame(useData, columns=useData[0]).drop([0]).rename(columns=useDict)
pd.set_option('display.max_colwidth', None)
useDF

In [ ]:
useDF = useDF.rename(columns={'1- or 5-character Import ENDUSE Code':'code'})

In [ ]:
enduse = useDF[(useDF.code =='0')|(useDF.code =='1')|(useDF.code =='2')|(useDF.code =='3')|(useDF.code =='4')|(useDF.code =='5')]
enduse

### Total Consumption by Product & Date

In [ ]:
enduseTOTAL = enduse[enduse['50-character District name'].str.contains("TOTAL")].drop(columns=['code','100-character Import ENDUSE Long Description','2-character District Code','Containerized Vessel Shipping Weight']).rename(columns={'100-character Import ENDUSE Short Description':'Product', '50-character District name': 'District', 'ISO-8601 Date/Time value':'Date','Imports for Consumption, Total Value':'Consumption Value'})
enduseTOTAL = enduseTOTAL.drop(columns='District')
enduseTOTAL

In [ ]:
useTCOL = enduseTOTAL.columns.drop(['Product',"Date"])
enduseTOTAL[useTCOL] = enduseTOTAL[useTCOL].apply(pd.to_numeric, errors='coerce')
enduseTOTAL.info()

In [ ]:
valuePD = enduseTOTAL.pivot(index="Date", columns='Product', values='Consumption Value')
valuePD

In [ ]:
valuePD.describe()

In [ ]:
valuePD.plot(kind='bar', stacked=True, figsize=(20,10))
plt.legend(bbox_to_anchor =(0.84,1.1), ncol = 3)
plt.show()

### Time Series International Trade: Monthly U.S. Imports by Port and Harmonized System (HS) Code
The Port HS endpoint in the Census data API also provides value, shipping weight, and method of transportation totals at the port level for all U.S. trading partners.

#### Variable

In [ ]:
port_var = pd.read_html('https://api.census.gov/data/timeseries/intltrade/imports/porths/variables.html')[0]
print(list(port_var['Name']))

In [ ]:
var_dict2 = dict(zip(port_var["Name"], port_var["Label"].str.replace('15-digit ', '')))

#### Data

In [ ]:
url2 = ("https://api.census.gov/data/timeseries/intltrade/imports/porths?get=")

In [ ]:
response2 = requests.get(f"{url2}I_COMMODITY,I_COMMODITY_LDESC,COMM_LVL,CNT_WGT_MO&time=2021&key={key}")
print(response2.url)

In [ ]:
df2 = response2.json()
df2 = pd.DataFrame(df2, columns=df2[0]).drop([0]).rename(columns=var_dict2)
df2.head()

In [ ]:
remove = df2[~df2["150-character Port Name"].str.contains("TOTAL")].drop(columns=['4-character Port Code','4-character Country Code'])
port = remove[~remove["50-character Country Name"].str.contains("TOTAL")].rename(columns={'150-character Port Name':'Port', '50-character District name': 'District', 'ISO-8601 Date/Time value':'Date'})
port.set_index('Port').head()

In [ ]:
import requests
import json
import prettytable

In [ ]:
bls_url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"

In [ ]:
bls_response = requests.get("")